<a href="https://colab.research.google.com/github/fangmarcy/Home_Sales/blob/main/Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [108 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [419 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [857 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,082 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,215 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [960 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http:

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)


In [7]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [8]:
print(df.columns)

['id', 'date', 'date_built', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view']


In [10]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

In [11]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
# SQL query to calculate the average price for a four-bedroom house sold in each year
query = """
    SELECT
        year(date) AS sale_year,
        ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY year(date)
    ORDER BY sale_year
"""

# Execute the query and get the result as a DataFrame
result = spark.sql(query)

# Show the result
result.show()


+---------+---------+
|sale_year|avg_price|
+---------+---------+
|     2019| 300263.7|
|     2020|298353.78|
|     2021|301819.44|
|     2022|296363.88|
+---------+---------+



In [12]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

# SQL query to calculate the average price of homes with 3 bedrooms and 3 bathrooms, for each year they were built
query = """
    SELECT
        date_built AS built_year,
        ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
    ORDER BY date_built
"""

# Execute the query and get the result as a DataFrame
result = spark.sql(query)

# Show the result
result.show()

+----------+---------+
|built_year|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [13]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
# SQL query to calculate the average price of homes with specific criteria, for each year they were built
query = """
    SELECT
        date_built AS built_year,
        ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3
        AND bathrooms = 3
        AND floors = 2
        AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY date_built
"""

# Execute the query and get the result as a DataFrame
result = spark.sql(query)

# Show the result
result.show()


+----------+---------+
|built_year|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [14]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
# Import the required libraries for measuring runtime
import time

# Define the price threshold
price_threshold = 350000

# Start the timer
start_time = time.time()

# SQL query to calculate the average price and "view" rating for homes greater than or equal to $350,000
query = f"""
    SELECT
        view,
        ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE price >= {price_threshold}
    GROUP BY view
"""

# Execute the query and get the result as a DataFrame
result = spark.sql(query)

# Show the result
result.show()

# Stop the timer and calculate the runtime
end_time = time.time()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

--- 0.4555180072784424 seconds ---


In [15]:
# 7. Cache the the temporary table home_sales.
spark.table("home_sales").cache()

DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [16]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [18]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
# Define the price threshold
price_threshold = 350000

start_time = time.time()

# SQL query to calculate the average price and filter out view ratings for homes greater than or equal to $350,000
query_cached = f"""
    SELECT
        view,
        ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE price >= {price_threshold}
    GROUP BY view
"""

# Execute the cached query and get the result as a DataFrame
result_cached = spark.sql(query_cached)

# Show the cached result
result_cached.show()

# Stop the timer for the cached query and calculate the runtime
end_time_cached = time.time()

print("--- %s seconds ---" % (time.time() - start_time))

# Start the timer for the uncached query
start_time_uncached = time.time()

# Re-run the same query on the uncached DataFrame (not using cache())
result_uncached = spark.sql(query_cached)

# Show the uncached result
result_uncached.show()

# Stop the timer for the uncached query and calculate the runtime
end_time_uncached = time.time()
runtime_uncached = end_time_uncached - start_time_uncached
print("Uncached query runtime:", runtime_uncached, "seconds")


+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

--- 0.27677440643310547 seconds ---
+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

Uncached query runtime: 0.32654738426208496 seco

In [19]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
# Define the output Parquet file path
output_parquet_path = "/path/to/parquet/output"

# Write the DataFrame to Parquet format with partitioning by "date_built"
df.write.partitionBy("date_built").parquet(output_parquet_path)

In [21]:
# 11. Read the parquet formatted data.
df = spark.read.parquet(output_parquet_path)

In [22]:
# 12. Create a temporary table for the parquet data.
df.createOrReplaceTempView("output_parquet_path")

In [25]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

price_threshold = 350000

start_time = time.time()

# SQL query to calculate the average price and filter out view ratings for homes greater than or equal to $350,000
query_parquet = f"""
    SELECT
        view,
        ROUND(AVG(price), 2) AS avg_price
    FROM output_parquet_path
    WHERE price >= {price_threshold}
    GROUP BY view
"""

# Execute the query on the Parquet DataFrame and get the result as a DataFrame
result_parquet = spark.sql(query_parquet)

# Show the result
result_parquet.show()

# Stop the timer for the query on Parquet DataFrame and calculate the runtime
end_time_parquet = time.time()

print("--- %s seconds ---" % (time.time() - start_time))

runtime_parquet = end_time_parquet - start_time
print("Query runtime on Parquet DataFrame:", runtime_parquet, "seconds")

+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

--- 0.4782111644744873 seconds ---
Query runtime on Parquet DataFrame: 0.4781198501586914 seconds


In [26]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [27]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")

# Print the result
if is_cached:
    print("'home_sales' DataFrame is cached.")
else:
    print("'home_sales' DataFrame is not cached.")


'home_sales' DataFrame is not cached.


In [28]:
spark.stop()